# An introduction to Coding Theory

$\textbf{Definition:}$ A $[n,k]$ - linear block code over $\mathbb{F}_{q}$ is a $k$ - dimensional linear subspace $\mathcal{C} \subseteq \mathbb{F}_{q}^{n}$.

We define an $\textit{encoding map}$ as: 

$$\varphi: \mathbb{F}_{q}^{k} \rightarrow \mathbb{F}_{q}^{n},$$ $$m \mapsto m\cdot G$$

which is represented by a $k \times n$ - matrix $G$. 

$G$ is called the $\textit{generator matrix}$ of $\mathcal{C}$ and the elements of $\mathcal{C}$  are called $\textit{codewords}$. We have $im(\varphi) = \mathcal{C}$. That is 

$$\begin{equation} \mathcal{C} = \text{rowspace}(G) \end{equation}.$$ 

Thus a generator matrix is a spanning matrix whose rows are linearly independent. We may easily construct many good codes using generator matrices. Of course it is not clear from the matrix how good the code will be. 




$\textbf{Example:}$ Let $$M = \begin{bmatrix}
 2 & 0 & 1 & 0 & 1 \\
 0 & 1 & 2 & 1 & 0 \\
 1 & 0 & 2 & 0 & 2 \\
\end{bmatrix} \in Mat_{3\times5}(\mathbb{F}_{3}).$$

We see that the first and the third row are linearly dependent. Our code can as well be generated by the full rank matrix 

$$M_{new} = \begin{bmatrix}
 1 & 0 & 2 & 0 & 2 \\
 0 & 1 & 2 & 1 & 0 \\
\end{bmatrix} \in Mat_{2\times5}(\mathbb{F}_{3}).$$

It is necessary to work with generator matrices which have a full rank.

In [ ]:
def get_SpanningMatrix(G,q):    
# Input: generator matrix of a code C, size q of finite field F_q 
# Output: generator matrix of the same code C which has full rank

    G = matrix(GF(q),G) # reduces G mod q
    rank = G.rank() 
    k = G.nrows()
    
    if rank != k: 
        RS = G.row_space() # finds rowspace of G
        basis = RS.basis() # put basis in a list
        G_new = matrix(basis) # makes new matrix out of the basis which has full rank
        return G_new 
    
    return G # returns original matrix if it has full rank from beginning

In [ ]:
M = matrix([[2,0,1,0,1],[0,1,2,1,0],[1,0,2,0,2]])
M_new = get_SpanningMatrix(M,3); M_new

Now for a given generator matrix $G$, we can generate all codewords of the code $\mathcal{C}$.

In [ ]:
def get_LinearCode(G,q):
# Input: generator matrix G, size q of finite field F_q 
# Output: list of codewords 
    
    G = get_SpanningMatrix(G,q)
    
    k = G.nrows()
    n = G.ncols()
    
    C = [m*G for m in GF(q)^k] # creates the codewords and put it in a list
    
    return C 

$\textbf{Example:}$ Let $G$ be the following generator matrix: 

$$G = \begin{bmatrix}
 1 & 0 & 1 & 0 & 1 & 1 \\
 0 & 1 & 1 & 1 & 1 & 0 \\
 0 & 0 & 0 & 1 & 1 & 1 \\
\end{bmatrix} \in Mat_{3\times6}(\mathbb{F}_{2}).$$

In [ ]:
G = matrix([[1,0,1,0,1,1],[0,1,1,1,1,0],[0,0,0,1,1,1]])

Lets see what codewords we get:

In [ ]:
get_LinearCode(G,2)

$\textbf{Definition:}$ We define the $\textit{weight}$ of a codeword $c = (c_1, c_2, \dots, c_n) \in \mathbb{F}_{q}^{n}$ as follows: 

$$\text{wt}(c) = \{ i \ | \ c_i \neq 0 \}.$$

In [ ]:
def get_weight(c): 
# Input: codeword as tuple, list or vector
# Output: weight of the codeword

    c = vector(c) 
    count = 0
    
    for i in range(len(c)):
        if c[i] != 0:
            count += 1 # counts the non-zero entries of a codeword
            
    return count

$\textbf{Example:}$ The codeword $c = (0,2,3,0,1,4) \in \mathbb{F}_{5}^{6}$ has weight 4.

In [ ]:
c = (0,2,3,0,1,4)
get_weight(c)

$\textbf{Definition:}$ The $\textit{Hamming distance}$ of two codewords $c,$ $d$ is defined by: 

$$d_H(c,d) = |\{ i \ | \ c_i \neq d_i \}|.$$

In [ ]:
def dist(c,d):
# Input: two codewords c, d
# Output: the distance between the two codewords

    c = vector(c)
    d = vector(d)
    
    assert(len(c) == len(d)) 

    count = 0
    for i in range(len(c)):
        if c[i] != d[i]:
            count += 1  # counts the non equal entries
                    
    return(count)

$\textbf{Example:}$ Let now $c = (2,1,5,0,2,0), \ d = (2,2,0,0,3,0)$ then the distance betweeen those codewords is 3.

In [ ]:
c = (2,1,5,0,2,0)
d = (2,2,0,0,3,0)
dist(c,d)

$\textbf{Definition:}$ The $\textit{distance}$ of a code $\mathcal{C}$ is given by: 

$$d(\mathcal{C}) = \min\{d_{H}(c,d) \ | \ c,d \in \mathcal{C}, \ c\neq d \}.$$ 

Since we are looking at linear codes it holds: 

$$d_{H}(c,d) = d_{H}(c-d,0) = \text{wt}(c-d) = \text{wt}(c').$$  
Thus we can compute the distance of a code more efficiently using: 

$$d(\mathcal{C}) = \min\{wt(c) \ | \ c \in \mathcal{C} \ , \ c \neq 0 \}.$$

In [ ]:
def get_distance(G,q):
# Input: generator matrix G, size q of finite field F_q
# Output: distance of the linear code

    C = get_LinearCode(G,q)
    dist = G.ncols() # maximal distance of the code
    
    for cw in C:
        if get_weight(cw) < dist and cw != 0: 
            dist = get_weight(cw) # finds the minimum weight of all codewords
            
    return dist

$\textbf{Example:}$ The distance of the code $\mathcal{C}$ which is generated by the matrix $G$ is:

In [ ]:
get_distance(G,2)

$\textbf{Lemma:}$ If $t \in \mathbb{N}$ and $d(\mathcal{C}) \geq 2t+1$ then up to $t$ errors can be corrected.

In [ ]:
def max_errors(G,q):
# Input: generator matrix G, size q of finite field F_q
# Output: maximal errors t which can be corrected 

    d = get_distance(G,q)
    t = floor((d-1)/2) # number of errors which can be corrected
    
    return t

$\textbf{Example:}$ For our given matrix $G$ we can correct maximal 1 error.

In [ ]:
max_errors(G,2)

# Decoding and correcting received codewords

Assume a message $m$ has to be sent and gets encoded to $c=mG$. After the transmission, we receive a word $y$. Then $y$ is of the form $y=mG + e$. If $e$ is the zero vector, then no error happend.  Our goal in this section is to implement a function, which checks if a received word $y$ is a valid codeword, i.e $e=(0,\dots,0)$. Afterwards we reconstruct the original message $m$. If $y$ is not a codeword, then an error occured. In that case $e\not = (0,\dots,0)$. If the weight of the error vector $e$ is small enough, our function will still be able to reconstruct $m$.

$\textbf{Definition:}$ We call the map: 

$$\psi: \mathbb{F}_{q}^{n} \rightarrow \mathbb{F}_{q}^{n-k},$$ 
$$y \mapsto H\cdot y^T$$

$\textit{syndrome former}$. 

It is represented by the $(n-k)\times n$ $\textit{parity check matrix}$ $H$. The function $\psi$ is defined such that 

$$\mathcal{C}= \text{ker}(\psi) = \{c\in\mathbb{F}_{q}^n\mid Hc^T=0\}.$$ 

If $H$ is a parity check matrix for $G$, then

$$ G \cdot H^T = H \cdot G^T = 0.$$

$\textbf{How to find the parity check matrix: }$

$\textbf{Definition:}$ A $k\times n$ - matrix $G$ is in $\textit{standard form}$ if it is of the form: 

$$G_{std} = [ \ Id_k \ | \ P \ ].$$ 

Let $G$ be a generator matrix of a code $\mathcal{C}$. Since $\mathcal{C}=\text{rowspace}(G)$, applying elementary row operations on $G$ does not change our code. If we can bring $G$ in standard form through these types of operations, we can easily find a parity check matrix by definining 

$$ H = [ \ \text{-}P^T \ | \ Id_{n-k} \ ]. $$

It is not always possible that we can obtain $G$ in standard form through row operations. The following example illustrates how we can still find a parity check matrix.


$\textbf{Example:}$ Consider again the generator matrix

$$G = \begin{bmatrix}
 1 & 0 & 1 & 0 & 1 & 1 \\
 0 & 1 & 1 & 1 & 1 & 0 \\
 0 & 0 & 0 & 1 & 1 & 1 \\
\end{bmatrix}. $$

Through row operations, we can bring $G$ in its row reduced echelon form:

$$G_{rref} = \begin{bmatrix}
 1 & 0 & 1 & 0 & 1 & 1 \\
 0 & 1 & 1 & 0 & 0 & 1 \\
 0 & 0 & 0 & 1 & 1 & 1 \\
\end{bmatrix}. $$

Now we swap column $3$ and $4$ to get:

$$\hat{G} = \begin{bmatrix}
 1 & 0 & 0 & 1 & 1 & 1 \\
 0 & 1 & 0 & 1 & 0 & 1 \\
 0 & 0 & 1 & 0 & 1 & 1 \\
\end{bmatrix}. $$

Note that in general $\hat{G}$ does not generate our original code. Since $\hat{G}$ is in standard form, we can get: 

$$\hat{H} = \begin{bmatrix}
 1 & 1 & 0 & 1 & 0 & 0 \\
 1 & 0 & 1 & 0 & 1 & 0 \\
 1 & 1 & 1 & 0 & 0 & 1 \\
\end{bmatrix}. $$

We have to swap the column $3$ and $4$ back to get the desired parity check matrix $H$ of $\mathcal{C}$:

$$H = \begin{bmatrix}
 1 & 1 & 1 & 0 & 0 & 0 \\
 1 & 0 & 0 & 1 & 1 & 0 \\
 1 & 1 & 0 & 1 & 0 & 1 \\
\end{bmatrix}. $$

In [ ]:
def get_StandardForm(G,q):
# Input: generator matrix G, size q of finite field F_q
# Output: list with matrix G in standard form and a tuple of swapped columns

    G = get_SpanningMatrix(G,q)

    n = G.ncols()
    k = G.nrows()
    G_rref = G.rref() # put G in row reduced echelon form
    G_columns = G_rref.columns() # creates a list with all columns of G_rref
    G_copy = copy(G_rref) # copies our matrix such that we can swap columns 
    Id = identity_matrix(k)
    swaps = [] # creates a list with swapped columns as tuples

    for i in range(k):
        if G_columns[i] != Id[i]:
            right_index = G_columns.index(Id[i]) # finds the index of the needed column
            swaps.append((i, right_index)) 
            G_copy.swap_columns(i,right_index) # swaps columns
            
    return [matrix(GF(q),G_copy), swaps]

In [ ]:
G = matrix([[1,0,1,0,1,1],[0,1,1,1,1,0],[0,0,0,1,1,1]]); G

In [ ]:
get_StandardForm(G,2)[0]

Here is the list of tuples which shows the permuted columns when we put $G$ in standard form:

In [ ]:
get_StandardForm(G,2)[1]

Note that we start counting from zero in SageMath, hence (2,3) represent the swap of column 3 and 4.

In [ ]:
def get_ParityCheckMatrix(G,q):
# Input: generator matrix G, size q of finite field F_q
# Output: corresponding parity check matrix H

    
    G = get_SpanningMatrix(G,q)
    
    k = G.nrows()
    n = G.ncols()
    if k >= n: 
        raise ValueError('This matrix does not generate a code') 
    
    G_standard = get_StandardForm(G,q)[0] 
    G_columns = G_standard.columns() 
    P_columns = []
    
    for i in range(n-k):
        col = G_columns[i+k] # get columns which doesn't belong to the Id of G_standard     
        P_columns.append(list(col)) # creats list with columns of the needed matrix P
    P = matrix(P_columns) # creates matrix P
    H = block_matrix([[-P.transpose()],[identity_matrix(n-k)]], subdivide=False).transpose() # H = [-P^T | Id_(n-k)]
    
    swaps = list(get_StandardForm(G,q)[1])
    l = len(swaps)
    for i in range(l):
        H.swap_columns(swaps[l-1-i][0],swaps[l-1-i][1]) # swaps all columns back
        
    return H

In [ ]:
H = get_ParityCheckMatrix(G,2); H

Given a parity check matrix $H$, we can now easily obtain a generator matrix $G$ of it:

In [ ]:
def get_GeneratorMatrix(H,q):
# Input: parity check matrix H, size q of finite field F_q
# Output: generator matrix G     
    
    return get_ParityCheckMatrix(H,q)

Let $H$ be a the parity check matrix from above,

$$H = \begin{bmatrix}
 1 & 1 & 1 & 0 & 0 & 0 \\
 1 & 0 & 0 & 1 & 1 & 0 \\
 1 & 1 & 0 & 1 & 0 & 1 \\
\end{bmatrix}.$$

We can check that $G \cdot H^{T} = 0$.

In [ ]:
G*H.transpose()

The following matrix is equivalent to our generator matrix $G$ from the example above.

In [ ]:
G_2 = get_GeneratorMatrix(H,2); G_2

Indeed the codewords are the same:

In [ ]:
C_1 = get_LinearCode(G,2); C_2 = get_LinearCode(G_2,2); 
sorted(C_1) == sorted(C_2)

The following function returns a list of all possible error vectors $e\in\mathbb{F}_q^n$ of weight $\text{wt}(e) \leq t$.

In [ ]:
def get_ErrorVectors(t,n,q):
# Input: Integer t, length n, size q of finite field F_q
# Output: vector of length n over F_q with weight smaller or equal t

    L = []
    L.append(zero_vector(n))
    tmp = copy(L)
    I = identity_matrix(n)
    count = 0
    tmp2 = []
    
    while(count < t):
        tmp2.clear()
        for e in tmp:
            for i in range(n):
                if e[i]!=0:
                    break 
                for element in GF(q):
                    if element!=0:
                        L.append(e + element*I.row(i))
                        tmp2.append(e + element*I.row(i))
        count += 1
        tmp.clear()
        tmp = copy(tmp2)   
        
    return L

For example, here is a list of all error vectors $e\in\mathbb{F}_5^6$ of weight $\text{wt}(e) \leq 1.$

In [ ]:
get_ErrorVectors(1,6,5)

$\textbf{Goal:}$ We want now to correct the received vector $y=mG + e$ and get the original codeword $c = y - e$ as well as the message $m$.

$\textbf{Idea:}$ Let $\mathcal{C}$ be a linear code with generator matrix $G$, parity check matrix $H$ and minimum distance $d$. Let $t=\lfloor\frac{d-1}{2}\rfloor$, then we may correct up to $t$ errors. Suppose $c=mG$ was sent, and up to $t$ errors occured, then $y=mG+e$ will be received. We assume $\text{wt}(e)\leq t$. It holds

$$ \begin{equation}
	Hy^T=H(c+e)^T=Hc^T+He^T=0+He^T=:s
\end{equation}$$


We can calculate $Hy^{T}$ to get the value $s=He^T$, which is called the $\textit{syndrom}$ of $e\in\mathbb{F}_{q}^n$. If $e$ is a codeword, its syndrome will be $0$. We keep a list of the syndromes of all $e\in\mathbb{F}_{q}^n$ with $\text{wt}(e)\leq t$. Then the corrected codeword will be $c = y-e$, where $e$ has syndrome $s$ from the list. Then we are looking for $m\in\mathbb{F}_{q}^k$ satisfying $mG=c$.

In [ ]:
def decode_received_cw(y,G,q):
# Input: received vector y, generator matrix G, size q of finite field F_q
# Output: corrected codeword x = y - error, original message m or a list of nearest (possible) codewords
    
    G = get_SpanningMatrix(G,q)
    
    n = G.ncols()
    y = matrix(y)
    H = get_ParityCheckMatrix(G,q)
    C = get_LinearCode(G,q)
    t = max_errors(G,q)
    s = H*y.transpose()
    errors = get_ErrorVectors(t,n,q)
    right_error = matrix([zero_vector(n)])
    
    for err in errors:
        if H*matrix(err).transpose() == s: # searching for the error
            right_error = matrix(err)
    x = y-right_error # decode the codeword
    
    if H*x.transpose() == 0: # checks if x is in C
        m = G.solve_left(x)
        print('Corrected codeword: {} --- Original message: {}'.format(x,m))
    else: 
        minimum = n 
        nearest_cw = []
        for cw in C:
            cw = matrix(cw)
            if dist(y,cw) < minimum:
                minimum = dist(y,cw) # finds smallest distance of a codeword and the received codeword
        for cw in C:
            if dist(y,cw) == minimum: # put the nearest codewords in a list
                nearest_cw.append(cw)
     
        print('Original message: Not recoverable --- Nearest codewords: {}'.format(nearest_cw))

$\textbf{Example:}$ Let $m=(0,0,1)$. Then $c=mG = (0,0,0,1,1,1)$. Now some error $e = (0,0,0,1,0,0)$ happened. The received codeword is $y = (0,0,0,0,1,1)$.

In [ ]:
y = (0,0,0,0,1,1)

In [ ]:
decode_received_cw(y,G,2)

# A few examples

Let $G_1 \in Mat_{3\times6}(\mathbb{F}_7)$ be the generator matrix of the code $\mathcal{C_1}.$

In [ ]:
G_1 = matrix([[0,6,2,1,4,4],[0,1,0,2,5,3],[4,0,1,0,4,0]]); print(G_1)
C_1 = get_LinearCode(G_1,7)
print()
print('Remark: G_1 has the distance {}, hence up to {} error can be corrected.'.format(get_distance(G_1,7), max_errors(G,7)))

Consider the codewords:

In [ ]:
x_1 = (0,6,2,1,4,4) # Message m = (1,0,0)
x_2 = (0,1,0,2,5,3) # Message m = (0,1,0)
x_3 = (4,0,1,0,4,0) # Message m = (0,0,1)

Now consider the received words: 

In [ ]:
y_1 = (0,6,2,1,4,4) # No error happend: y_1 = x_1 + (0,0,0,0,0,0)
y_2 = (0,5,0,2,5,3) # One error happend: y_2 = x_1 + (0,4,0,0,0,0)
y_3 = (4,0,2,0,4,3) # Two errors happend: y_3 = x_3 + (0,0,1,0,0,3)

For $y_1$ we see that the same codeword gets returned, since no error happend.

In [ ]:
decode_received_cw(x_1,G_1,7)

We can correct $y_2$ to $x_2$, since $\text{wt}(e_2) = 1$.

In [ ]:
decode_received_cw(y_2,G_1,7)

We can not decode $y_3$ uniquely since $\text{wt}(e_3) = 2 > 1$. It will return a list with possible codewords which have the smallest distance to the received one.

In [ ]:
decode_received_cw(y_3,G_1,7)

Our function also works for finite fields which are not of prime order. 

In [ ]:
M = MatrixSpace(GF(4),2,5) 
G_2 = M.random_element(); G_2 # picks a random 2x5 matrix with entries in the finite field F_4 

Let now $y_4 \in \mathbb{F}_{4}^{5}.$

In [ ]:
y_4 = (GF(4)^5).random_element()
decode_received_cw(y_4,G_2,4)

# Test function

We want to test our function for random generator matrices and received vectors.

In [ ]:
def test_decoding(q,k,n):
# Input: q size of the finite field F_q, k number of rows of G, n number of columns of G
# Output: generator matrix G, received vector y, decoded codeword or a list of possible codewords

    M = MatrixSpace(GF(q),k,n) 
    G_test = M.random_element() # pick a random kxn - matrix with entries in F_q
    y_test = (GF(q)^n).random_element() # pick a random vector of length n with entires in F_q
    print('Generator matrix:')
    print(G_test) 
    print()
    print('Received vector:')
    print(y_test) 
    print()
    
    return decode_received_cw(y_test,G_test,q) 

$\textbf{Example:}$ Let $G_{test} \in Mat_{k\times n}(\mathbb{F}_{q})$ and $y_{test} \in \mathbb{F}_{q}^{n}.$ Consider random variables $q, k, n$ and test the function.

In [ ]:
test_decoding(3,2,6)

# Hamming Codes

The $\textit{Hamming code}$ is a single error correction linear block code with $\left[n,k\right] = \left[q^m-1,q^m-1-m\right]$, where $m=n-k$ is the number of check bits in the codeword. The simplest non-tirivial code is $m=3$, which is the $\left[7,4\right]$-Hamming code over $\mathbb{F}_2$. Its parity check matrix is given by $\newline$ $$ H = \begin{equation} \begin{bmatrix} 0& 0&1&1&0&1&1\\0&1&0&1&1&1&0\\1&1&1&1&0&0&0 \end{bmatrix} \end{equation}.$$

By definition, Hamming codes can correct up to one error. Therefore the distance must be $d(\mathcal{C})= 3$  or $4$. Recall that $\newline$ $$d(\mathcal{C}) = \min\{\text{wt}(c) \ | \ c \in \mathcal{C} \ ,\ c \neq 0 \}.$$ 
$\newline$ There is also a way to calculate the distance of $\mathcal{C}$ by just looking at its parity check matrix. 

$\textbf{Theorem:}$Let $d(\mathcal{C})=d$. Then $d$ is the minimum number, such that any $d-1$ columns of the parity check matrix $H$ of $\mathcal{C}$ are linearly independent. 

From this theorem we can conclude that Hamming codes have a minimum distance of $3$, since any two columns of its parity check matrix are linearly independent.

Now we show how we can construct Hamming codes for any $m\in\mathbb{N}$ over any field $\mathbb{F}_{q}$. Consider the projective space $\mathbb{P}^{m}_{\mathbb{F}_q}=\{v\in\mathbb{F}_{q}^{m+1}\setminus\{0\}\}\big/{\sim}$ where $v\sim w:\iff \exists\lambda\in\mathbb{F}_q^{\star}$ with $w=\lambda v$. Thus by taking each representative of the projective space as a column for our parity check matrix $H$, we get a Hamming code of distance $d(\mathcal{C})=3$, since any two columns are linearly independent by definition of the equivalence relation.


In [ ]:
def IntegerMod_to_Int(L): # Helper function
# Input: list with entries over Z_p
# Output: list with same entries over Z
    
    S = []
    for i in range(len(L)):        
        m = Integer(L[i])
        S.append(m)        
    return S 


def create_Hamming_code(m, q=2):
# Input: natural number m
# Output: generator matrix H of a Hamming code
    
    X = ProjectiveSpace(m-1, GF(q))
    columns = []
    for p in X.rational_points():
        v = []
        for i in range(len(p)):
            v.append(p[i])
        columns.append(v)    
    
    H = matrix(ZZ, columns[0])
    for j in range(1, len(columns)):
        col = IntegerMod_to_Int(columns[j])
        H = H.insert_row(j,col)
    
    H = H.transpose()
    H = matrix(GF(q), H)
    
    return H

In [ ]:
H = create_Hamming_code(3,2)
H

# Cyclic Codes

$\textbf{Definition:}$ A linear code $\mathcal{C}$ is called $\textit{cyclic}$, if for any codeword $c =(c_1,c_2,\dots,c_n)\Longrightarrow c_{\text{shift}} = (c_n,c_1,\dots c_{n-1})\in\mathcal{C}.$ 

Consider the following Code $\mathcal{C}$ generated by the following matrix $G$:

In [ ]:
G = matrix(GF(2), [[1,0,0,1,1,0,0,1],[0,1,0,0,1,0,1,1],[1,0,1,0,1,0,1,0], [1,0,1,1,0,1,0,0]]); G

Recall that $\mathcal{C}$ is generated by the rows of its generator matrix $G$. Therefore, we have for example the codeword $c = (0,1,0,0,1,0,1,1)$. Let's look at its cyclic shift $c_{\text{shift}} = (1,0,1,0,0,1,0,1)$.

In [ ]:
def shift_code(c):
# Input: codeword c
# Output: shifted codeword
    
    c = list(c)
    i = c.pop(len(c)-1)
    c.insert(0,i)

    return vector(c)

In [ ]:
c = (0,1,0,0,1,0,1,1)
c_shift = shift_code(c)
c_shift

In [ ]:
C = get_LinearCode(G,2)
print(vector(c) in C)
print(vector(c_shift) in C)

We conclude that $C$ is not cyclic. Our next goal is to construct cyclic codes of length $n$ over any prime field $\mathbb{F}_p$. For that, we use the following theorem:

$\textbf{Theorem:}$ Let $g(x)\in\mathbb{F}_p\left[x\right]$ such that $g(x)\mid x^n-1$. If $g(x) = g_0+g_1x+\dots+g_rx^r$, then 

\begin{equation} G = \begin{bmatrix} g_0 & g_1 & \dots & g_r & 0 & 0 & \dots & 0 \\ 0 & g_0 & g_1 & \dots & g_r & 0 & \dots & 0 \\ \vdots &  \ \ \ \ddots & \ & \ & \ &\ &\ \\ 0 & \dots & g_0 & g_1 &\dots & \ & &g_r \end{bmatrix} \in\mathbb{F}_q^{(n-r)\ \times\ n}\end{equation}

is a generator matrix of a $[n,n-r]$ - cyclic code. 

Hence, in order to construct a cyclic code of length $n$ over $\mathbb{F}_p$ we must find the divisors of the polynomial $x^n-1\in\mathbb{F}_p\left[x\right]$.

In [ ]:
def get_CyclicCode(n,p, div = 1):
# Input: integer n, p prime, optional: divisor of x^n - 1 
# Output: div = 1 : list of all cyclic codes of length n over F_q
#         div =! 1 : generator matrix of a cyclic code generated by the divisor polynomial
    
    assert(p.is_prime())
    
    R.<x> = PolynomialRing(GF(p))
    L = []
    P = x^n - 1
    F = factor(P)  
    
    # finding all possible non trivial divisors of P = x^n - 1
    div = R(div)
    S = set(F) 
    Divisors = [] 
    PS = subsets(F)
    for A in PS:
        prod = 1
        for g in A:
            prod = prod * g[0]
        Divisors.append(prod)      
    
    assert(div in Divisors) # checking if optional argument is indeed a divisor of x^n - 1
    Divisors.remove(1) # corresponds to trivial code C = F_p
    Divisors.remove(P) # corresponds to trivial code C = {0}
    
    # Computing generator matrix of the cyclic code generated by the divisor    
    # Returns corresponding generator matrix 
    
    if div in Divisors and P!=1:
        r = div.degree()
        col = div.coefficients(sparse=False) + [0 for l in range(n-r-1)] # first row of G
        G = matrix(ZZ,col)        
        for k in range(n-r-1):
            col = copy(col)
            col = shift_code(col) # shifts the row to the right
            col = IntegerMod_to_Int(col) # Need to do this since G with entries in F_q has no attribute insert_row()           
            G = G.insert_row(k+1,col)

        return G
    
    # Compumptes the generator matrices for every cyclic codes of length n
    # Returns list of the generator matrices
    
    for f in Divisors:     
        r = f.degree()
        col = f.coefficients(sparse=False) + [0 for l in range(n-r-1)] # first row of G
        G = matrix(ZZ,col)        
        for k in range(n-r-1):
            col = copy(col)
            col = list(shift_code(col)) # shifts the row to the right
            col = IntegerMod_to_Int(col) # Need to do this since G with entries in F_q has no attribute insert_row()           
            G = G.insert_row(k+1,col)
        L.append(G) 
        
    return L

For instance, let us create a cyclic code of length $9$ over $\mathbb{F}_{5}$. 

In [ ]:
P.<z> = PolynomialRing(GF(5))
f = z**9 - 1 
f.factor()

Consider $g(z) = z^6 + z^3 + 1$. By the last theorem, the corresponding generator matrix looks like 

$$ \begin{equation} G = \begin{bmatrix} 1&0&0&1&0&0&1&0&0 \\ 0&1&0&0&1&0&0&1&0 \\ 0&0&1&0&0&1&0&0&1 \end{bmatrix} \end{equation}.$$

In [ ]:
g = z^6 + z^3 + 1
G = get_CyclicCode(9,5,g)
G

Indeed, the last matrix defines a cyclic code:

In [ ]:
C = get_LinearCode(G,5)
n = len(C)
r = randrange(n)
c = C[r] #random codeword
c_shift = shift_code(c)

c_shift in C

Alternatively, by not using the optional argument of the function $\textit{get_CyclicCode(n,q, div = 1)}$, we get a list of all cyclic codes of length $n$ over $\mathbb{F}_{q}$. As another example, let's search for all cyclic codes of length $15$ over $\mathbb{F}_7$.

In [ ]:
P.<z> = PolynomialRing(GF(7))
f = z^15 - 1
f.factor()

We see that $z^{15}-1$ splits into $6$ distinct irreducible factors over $\mathbb{F}_7$. Hence there are $2^6 - 2$ non-trivial divisors of $z^{15}-1.$

The above theorem implies that there are 62 non-trivial cyclic codes of length $15$ in $\mathbb{F_7}$. 

In [ ]:
cyclic_codes = get_CyclicCode(15,7)
len(cyclic_codes)

In [ ]:
cyclic_codes

# MDS Codes - Reed Solomon Code

$\textbf{Question:}$ Among all $\left[n,k\right]$ - codes $\mathcal{C}\subseteq\mathbb{F}_q^n$, what is the maximal possible distance?

$\textbf{Theorem:}$ (Singleton Bound) $$\begin{equation} d(\mathcal{C})\leq n-k+1\end{equation}.$$

$\textbf{Definition:}$ An $\left[n,k\right]$ - linear code $\mathcal{C}\subset\mathbb{F}_q^n$ is called a $\textit{maximum seperable [MDS]}$ code if $d(\mathcal{C})=n-k+1$.

$\textbf{Theorem:}$ $\mathcal{C}$ is MDS. $\iff$ For any parity check matrix, $n-k$ columns are linearly independent.

Our goal is to construct arbitrary $[n,k]$ - MDS codes. We proceed as follows:

$\textbf{Construction:}$ Assume the field size is $|\mathbb{F}_q|=q\geq n$. Let $\{\alpha_1,\dots,\alpha_n\}\subset\mathbb{F}_q$ be pairwise different elements. First construct the following $(n-k)\times n$ matrix:

$$ H =\begin{bmatrix}
	1 & 1 & \dots & 1 \\ \alpha_1 & \alpha_2 & \dots & \alpha_n \\ \vdots&\vdots&&\vdots \\ \alpha_1^{n-k-1}&\alpha_2^{n-k-1}&\dots&\alpha_n^{n-k-1}
\end{bmatrix}.$$

We notice that all $(n-k)\times(n-k)$ submatrices are Vandermonde matrices. Since we chose $\alpha_1,\dots,\alpha_n$ to be pairwise different, all the corresponding minors are non-zero. By the theorem above, the obtained code is MDS.


Specifically, suppose we want to construct a three-error-correcting MDS code $\mathcal{C}$ over $\mathbb{F}_{23}$. Therefore our code must have distance $7$. Let us choose $n=9$ and $k=3$. Then $7 = 9 - 3 + 1$.

Choose $9$ distinct elements in $\mathbb{F}_{23}$:

In [ ]:
elements = [3,17,2,9,7,6,11,20,22]

Now we create our generator matrix:

In [ ]:
def createReedSolomonCode(gen,k,q):
#Input: pairwise distinct elements gen, dimension k of the code, q size of the finite field F_q
#Output: Reed Solomon code
    
    n = len(gen)
    H = matrix(GF(q),[[gen[j]^i for j in range(n)] for i in range(n-k)])
    
    return H

In [ ]:
L = createReedSolomonCode(elements,3, 23); L
L 

Let's see if the distance of our code is indeed 7:

In [ ]:
G = get_GeneratorMatrix(L,23)
get_distance(G,23)